# Preenchimento Proativo
Mostra o comportamento da rede utilizando o preenchimento proativo. Essa simulação registra os recursos utilizados:

    - Número de Qubits criados
    - Número de EPRs criados

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_consumption, clear_file
from quantumnet.objects import time

Informações para a simulação

In [2]:
# Caminho do arquivo de saída
path = "resultados/proativo/qubits_eprs_sync.csv"
# Limpa o arquivo de saída
clear_file(path)
# Sobre a rede
rede = Network()
rede.n_initial_eprs = 10
rede.n_initial_qubits = 10
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Sobre o controlador
controlador = Controller(rede)
controlador.default_ttl = 100
# Sobre as requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

Preenchimento proativo

In [3]:
hosts = rede.hosts
for alice in hosts:
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.5, 0.6), 5)
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.8, 0.9), 5)

Funções responsáveis pela passagem de tempo

In [4]:
def update_rules(host, time_for_update):
    """
    Atualiza as regras de um host
    Args:
        host (Host): Host que terá as regras atualizadas
        time_for_update (int): Tempo para atualização das regras
    """
    # Percorre a tabela de fluxo do host
    for match in host.flow_table.copy():
        # Recebe o objeto da regra, somente
        rule = host.flow_table[match][1]
        # Incrementa a idade da regra
        for t in range(time_for_update):
            rule.increment_age()
        # Verifica se a regra expirou
        if rule.ttl <= rule.age:
            if rule.hit_count == 0:
                host.flow_table.pop(match)
                print(f"[Time {time.get_current_time()}] Regra {rule} expirada em {host}")
            else:
                print(f"Regra reutilizada {rule} em {host}")
                rule.hit_count = 0
                rule.age = rule.ttl/2

def update_time(time_for_update):
    """
    Atualiza o tempo e as regras

    Args:
        time_for_update (int): Tempo para incrementar o time-slot.
    """
    # Atualiza o tempo
    for t in range(time_for_update):
        time.increment()
        
        # Registra o consumo de recursos
        register_consumption(time.get_current_time(), rede.registry_of_resources, path)
        
        # Atualiza os recursos de 10 em 10
        if time.get_current_time() % 10 == 0:
            rede.refresh_resources(num_qubits=10, num_eprs=10)
            print(f"[Time {time.get_current_time()}] Recursos atualizados")
    # Atualiza as regras
    for host in rede.hosts:
        update_rules(rede.get_host(host), time_for_update)

Preenchimento reativo, baseado nas requests.

    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.

In [5]:
for request in requests:
    print(f"[Time {time.get_current_time()}]")
    print(f"Request: {request}, Alice: {request.alice}, Bob: {request.bob}, Fmin: {request.fmin}, Neprs: {request.neprs}")
    alice = rede.get_host(request.alice)
    rule = alice.find_rule_by_request(request)

    if rule == False:  # Caso não exista um match na tabela
        print(f"Descartando requisição {request} em {alice}")
        request.endtime = time.get_current_time()
        # Registra no CSV como um novo registro
    else:  # Caso já exista a regra
        request.starttime = time.get_current_time()
        controlador.run_rule(rule[1])
        request.endtime = time.get_current_time() + 1
    
    update_time(1)
    # Exibir informações da requisição
    print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

[Time 0]
Request: R-417710, Alice: 4, Bob: 1, Fmin: 0.54, Neprs: 3
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(4-0)
[BasicRuleProactive] Ação: CreateEPR(0-1)
[BasicRuleProactive] Passo: 2
[BasicRuleProactive] Ação: Swap(4-0-1)
Running Swap(4-0-1)
Time-slot 1 registrado: Qubits = 120, EPRs = 172
Request R-417710: Start Time = 0, End Time = 1
[Time 1]
Request: R-056BD0, Alice: 5, Bob: 0, Fmin: 0.83, Neprs: 3
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(5-4)
[BasicRuleProactive] Ação: CreateEPR(4-0)
[BasicRuleProactive] Passo: 2
[BasicRuleProactive] Ação: Swap(5-4-0)
Running Swap(5-4-0)
Time-slot 2 registrado: Qubits = 120, EPRs = 174
Request R-056BD0: Start Time = 1, End Time = 2
[Time 2]
Request: R-4177A0, Alice: 7, Bob: 10, Fmin: 0.92, Neprs: 4
Descartando requisição R-4177A0 em 7
Time-slot 3 registrado: Qubits = 120, EPRs = 174
Request R-4177A0: Start Time = 0, End Time = 2
[Time 3]
Request: R-417830, Alice: 0, Bob: 5, Fmin: 0.62, Neprs: 3
